In [1]:
import numpy as np, pandas as pd
import statsmodels.api as sm
from ISLP.models import (ModelSpec as MS,
                         summarize,)
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP import confusion_table
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [2]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

In [3]:
from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary

In [4]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger


In [5]:
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)

Seed set to 0


In [6]:
from torchvision.io import read_image
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50,
                                ResNet50_Weights)
from torchvision.transforms import (Resize,
                                    Normalize,
                                    CenterCrop,
                                    ToTensor)

In [7]:
from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)

In [8]:
from ISLP.torch.imdb import (load_lookup,
                             load_tensor,
                             load_sparse,
                             load_sequential)

In [9]:
from glob import glob
import json

In [10]:
from sklearn.cluster import \
     (KMeans,
      AgglomerativeClustering)
from scipy.cluster.hierarchy import \
     (dendrogram,
      cut_tree)
from ISLP.cluster import compute_linkage
from sklearn.decomposition import PCA


# FC-NN on Default Dataset

In [11]:
Default = load_data('Default').dropna()

In [12]:
Default

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [13]:
Default['student'] = Default['student'].map({'Yes': 1, 'No': 0})
Default['default'] = Default['default'].map({'Yes': 1, 'No': 0})

In [14]:
model = MS(Default.columns.drop('default'), intercept=False)
X = model.fit_transform(Default)
Y = Default['default']


In [15]:
(X_train, 
 X_test,
 Y_train,
 Y_test) = train_test_split(X,
                            Y,
                            test_size=1/3,
                            random_state=1)

In [16]:
glm_train = sm.GLM(Y_train,
                   X_train,
                   family=sm.families.Binomial())

results = glm_train.fit()
summarize(results)

,coef,std err,z,P>|z|
student,-3.8195,0.185000,-20.665,0.0
balance,0.0028,0.000000,18.633,0.0
income,-0.0002,0.000006,-27.814,0.0


In [17]:
Y_test = Y_test.map({1: 'Yes', 0: 'No'})

In [18]:
# Rezultati linearne logističke regresije
probs = results.predict(exog=X_test)
labels = np.array(['Yes']*len(X_test))
labels[probs<0.5] = 'No'  
print(accuracy_score(Y_test,
                     labels))
confusion_table(labels, Y_test)

0.9643071385722856


Truth,No,Yes
Predicted,,
No,3205,90
Yes,29,10


In [19]:
class DefaultModel(nn.Module):

    def __init__(self, input_size):
        super(DefaultModel, self).__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(
            nn.Linear(input_size, 10),  
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(10, 2))  

    def forward(self, x):
        x = self.flatten(x)
        return self.sequential(x)

In [20]:
default_model = DefaultModel(X.shape[1])

In [21]:
summary(default_model, 
        input_size=X_train.shape,
        col_names=['input_size',
                   'output_size',
                   'num_params'])


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
DefaultModel                             [6666, 3]                 [6666, 2]                 --
├─Flatten: 1-1                           [6666, 3]                 [6666, 3]                 --
├─Sequential: 1-2                        [6666, 3]                 [6666, 2]                 --
│    └─Linear: 2-1                       [6666, 3]                 [6666, 10]                40
│    └─ReLU: 2-2                         [6666, 10]                [6666, 10]                --
│    └─Dropout: 2-3                      [6666, 10]                [6666, 10]                --
│    └─Linear: 2-4                       [6666, 10]                [6666, 2]                 22
Total params: 62
Trainable params: 62
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.41
Input size (MB): 0.08
Forward/backward pass size (MB): 0.64
Params size (MB): 0.00
Estimated Total Size (MB): 0.72

In [22]:
Y_test = Y_test.map({'Yes': 1, 'No': 0})

In [23]:
X_train_t = torch.tensor(X_train.to_numpy().astype(np.float32))
Y_train_t = torch.tensor(Y_train.to_numpy().astype(np.int64))  
default_train = TensorDataset(X_train_t, Y_train_t)

In [24]:
X_test_t = torch.tensor(X_test.to_numpy().astype(np.float32))
Y_test_t = torch.tensor(Y_test.to_numpy().astype(np.int64))   
default_test = TensorDataset(X_test_t, Y_test_t)

In [25]:
max_num_workers = rec_num_workers() # = 4

In [26]:
default_dm = SimpleDataModule(default_train,
                          default_test,
                          batch_size=32,
                          num_workers=min(4, max_num_workers),
                          validation=default_test)

In [27]:
#cross-entropy loss function
default_module = SimpleModule.classification(default_model,
                                           num_classes=2)
default_logger = CSVLogger('logs', name='Default')

In [28]:
default_trainer = Trainer(deterministic=True,
                      max_epochs=50,
                      log_every_n_steps=5,
                      logger=default_logger,
                      callbacks=[ErrorTracker()])
default_trainer.fit(default_module, datamodule=default_dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type             | Params
-------------------------------------------
0 | model | DefaultModel     | 62    
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
62        Trainable params
0         Non-trainable params
62        Total params
0.000     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 209/209 [00:05<00:00, 35.84it/s, v_num=12]        
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  96%|█████████▌| 101/105 [00:00<00:00, 243.19it/s]


Epoch 1: 100%|██████████| 209/209 [00:01<00:00, 137.03it/s, v_num=12]      
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  93%|█████████▎| 98/105 [00:00<00:00, 274.91it/s]


Epoch 2: 100%|██████████| 209/209 [00:01<00:00, 177.28it/s, v_num=12]      
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  90%|████████▉ | 94/105 [00:00<00:00, 313.40it/s]


Epoch 3: 100%|██████████| 209/209 [00:01<00:00, 174.11it/s, v_num=12]      
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  87%|████████▋ | 91/105 [00:00<00:00, 328.03it/s]


...

Epoch 48: 100%|██████████| 209/209 [00:01<00:00, 182.56it/s, v_num=12]     
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  35%|███▌      | 37/105 [00:00<00:00, 338.36it/s]


Epoch 49: 100%|██████████| 209/209 [00:01<00:00, 160.10it/s, v_num=12]     
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  31%|███▏      | 33/105 [00:00<00:00, 301.79it/s]


Epoch 49: 100%|██████████| 209/209 [00:01<00:00, 126.74it/s, v_num=12]     

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 209/209 [00:01<00:00, 126.74it/s, v_num=12]


In [29]:
default_trainer.test(default_module, datamodule=default_dm) ## Testiranje na testnom skupu


Testing DataLoader 0: 100%|██████████| 105/105 [00:00<00:00, 420.10it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          0.970005989074707
        test_loss           0.13516245782375336
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.13516245782375336, 'test_accuracy': 0.970005989074707}]

In [30]:
## 0.13521292805671692 + 0.970005989074707 ne mora biti 1, 0.135 je entropija!

Neuralna mreža točnost: 0.97

Logistička točnost: 0.96
#
Neural network accuracy: 0.97

Logistic regression accuracy: 0.96

In [31]:
del(Default,
    default_model, default_dm,
    default_logger,
    default_test, default_train,
    X, Y,
    X_test, X_train,
    Y_test, Y_train,
    X_test_t, Y_test_t,
    default_trainer, default_module)